In [35]:
import os
import random
import pickle
from collections import Counter

import datasets
import pandas as pd
import numpy as np

from datasets import load_from_disk, interleave_datasets
from nltk import sent_tokenize

In [8]:
root_dir = "/work/ogalolu/datasets"

# Initial dataset to create seperate lines for each article item for available languages.
path = os.path.join(root_dir, "filtered_small_updated")
filtered_small_updated = load_from_disk(path)
print(len(filtered_small_updated))

languages = ('fr', 'it', 'de', 'en')

1274091


In [7]:
filtered_small_updated[0]

{'text_it': None,
 'text_fr': None,
 'wikidata_id': 'Q3745033',
 'text_de': '  Karriere  Hlúpik begann seine Karriere beim 1. FC Slovácko. Sein Debüt für Slovácko in der höchsten tschechischen Spielklasse gab er im Mai 2010, als er am 28. Spieltag der Saison 2009/10 gegen Sparta Prag in der 85. Minute für Radim Nečas eingewechselt wurde. Seinen ersten Treffer für Slovácko erzielte er im Juli 2011 gegen den 1. FK Příbram. Nach fast sieben Jahren in Slováckos erster Mannschaft verließ Hlúpik den Verein in der Winterpause der Saison 2016/17 und wechselte nach Bulgarien zu Tscherno More Warna. Bis zu seinem Wechsel war er in 96 Erstligapartien eingesetzt worden und hatte dabei neun Tore erzielt. Für Tscherno More Warna absolvierte er in der Rückrunde der Saison 2016/17 zwei Partien in der regulären Saison und zwei Partien in der Meisterschaftsrunde. Im August 2017 wechselte er nach Finnland zum IFK Mariehamn, bei dem er bereits vor seinem Wechsel nach Bulgarien ein Probetraining absolviert

## Long Dataset

In [34]:
def make_long(example, languages: tuple, min_sentences: int):
    available_list = list()
    new_example = dict()

    for language in languages:
        if example[f"text_{language}"][0] and len(sent_tokenize(example[f"text_{language}"][0])) >= min_sentences:
            available_list.append(language)

    articles = list()

    for language in available_list:
        articles.append(*example[f"text_{language}"])

    new_example["wikidata_id"] = example["wikidata_id"] * len(available_list)  
    new_example["language"] = available_list
    new_example["article"] = articles

    return new_example

# Filter the articles 
min_sentences = 3

long_dataset = filtered_small_updated.map(make_long, fn_kwargs={"languages": languages, "min_sentences": min_sentences}, 
                                    remove_columns=filtered_small_updated.column_names, 
                                    batched=True, 
                                    batch_size=1)
print(f"Number of articles in the dataset: {len(long_dataset)}")

  0%|          | 0/1274091 [00:00<?, ?ba/s]

Number of articles in the dataset: 2956942


In [37]:
long_dataset[0]

{'wikidata_id': 'Q3745033',
 'article': '  Karriere  Hlúpik begann seine Karriere beim 1. FC Slovácko. Sein Debüt für Slovácko in der höchsten tschechischen Spielklasse gab er im Mai 2010, als er am 28. Spieltag der Saison 2009/10 gegen Sparta Prag in der 85. Minute für Radim Nečas eingewechselt wurde. Seinen ersten Treffer für Slovácko erzielte er im Juli 2011 gegen den 1. FK Příbram. Nach fast sieben Jahren in Slováckos erster Mannschaft verließ Hlúpik den Verein in der Winterpause der Saison 2016/17 und wechselte nach Bulgarien zu Tscherno More Warna. Bis zu seinem Wechsel war er in 96 Erstligapartien eingesetzt worden und hatte dabei neun Tore erzielt. Für Tscherno More Warna absolvierte er in der Rückrunde der Saison 2016/17 zwei Partien in der regulären Saison und zwei Partien in der Meisterschaftsrunde. Im August 2017 wechselte er nach Finnland zum IFK Mariehamn, bei dem er bereits vor seinem Wechsel nach Bulgarien ein Probetraining absolviert hatte. Für den Verein aus Åland kam

## Sampling

In [39]:
# Language pair distribution within the dataset
lang_dict = Counter(long_dataset["language"])
print(lang_dict.items())

dict_items([('de', 806805), ('en', 1000203), ('fr', 680050), ('it', 469884)])


In [40]:
# For Counter class, order of returned list of keys/values doesn't change

language_datasets = []
for lang in lang_dict.keys():
    language_dataset = long_dataset.filter(lambda example: example["language"] == lang, num_proc=8)
    language_datasets.append(language_dataset)

In [41]:
#
def prob_with_temp(array: np.array, temperature: float = 0.3) -> np.array:

    prob_array = array / sum(array)
    print(f" Distribution before temperature adjustment: {prob_array}")
    final_array = prob_array **  temperature / sum(prob_array **  temperature)
    print(f" Distribution after temperature adjustment: {final_array}")
    return final_array

In [42]:
initial_array = np.array(list(lang_dict.values()))
final_array = prob_with_temp(initial_array)

 Distribution before temperature adjustment: [0.27285114 0.33825587 0.22998422 0.15890876]
 Distribution after temperature adjustment: [0.25860557 0.27582507 0.24567986 0.2198895 ]


## Merging and Saving

In [43]:
final_dataset = interleave_datasets(language_datasets, probabilities=final_array, seed=42)

In [44]:
final_pair_dict = Counter(final_dataset["language"])
print(final_pair_dict.items())

dict_items([('fr', 523895), ('en', 589614), ('it', 469884), ('de', 552171)])


In [45]:
final_dataset = final_dataset.flatten_indices()

  0%|          | 0/2136 [00:00<?, ?ba/s]

In [ ]:
path = os.path.join(root_dir, "longformer_small")
final_dataset.save_to_disk(path)